In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, brier_score_loss
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

In [3]:
df_dev = pd.read_csv('Final_Output_Class_dev.csv')
df_test = pd.read_csv('Final_Output_Class_test.csv')
#SPlit the label
label_dev = df_dev['Underpriced']
label_test = df_test['Underpriced']
#Drop the label column
df_dev = df_dev.drop(columns='Underpriced')
df_test = df_test.drop(columns='Underpriced')

In [4]:
x_train,x_dev,y_train,y_dev = train_test_split(df_dev,label_dev,test_size=0.2,random_state=42)

In [ ]:
rf = RandomForestClassifier(random_state=42)
#Use grid search for the optimal hyperparameter
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500, 600],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=2
)
grid_search.fit(x_train,y_train)

best_rf = grid_search.best_estimator_
y_dev_pred = best_rf.predict(x_dev)
best_f1 = f1_score(y_dev, y_dev_pred)

print(f"Best F1 Score on Dev Set: {best_f1:.4f}")
print("Best Hyperparameters:")
print(grid_search.best_params_)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits
✅ Best F1 Score on Dev Set: 0.8982
🏆 Best Hyperparameters:
{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}


In [24]:
test_pred= best_rf.predict(df_test)
#Compute Training Accuracy
test_accuracy = best_rf.score(df_test,label_test)
#Compute F1 score
test_f1 = f1_score(label_test, test_pred)

print(f'Test Accuracy :{test_accuracy}')
print(f'Test F1 Score :{test_f1}')

Test Accuracy :0.8077889447236181
Test F1 Score :0.2233502538071066


In [8]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'linear']
}

grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

y_dev_pred = best_model.predict(x_dev)
f1 = f1_score(y_dev, y_dev_pred)

print("Best SVM Params:", grid_search.best_params_)
print("Best SVM F1 Score on Dev Set:", f1)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best SVM Params: {'C': 10, 'gamma': 0.01, 'kernel': 'poly'}
Best SVM F1 Score on Dev Set: 0.8415492957746479


In [9]:
test_pred= best_model.predict(df_test)
#Compute Training Accuracy
test_accuracy = best_model.score(df_test,label_test)
#Compute F1 score
test_f1 = f1_score(label_test, test_pred)

print(f'Test Accuracy :{test_accuracy}')
print(f'Test F1 Score :{test_f1}')



Test Accuracy :0.6695979899497487
Test F1 Score :0.3538083538083538
